#Set-Up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

#Scheduling Problem

In [ ]:
#inputs
num_workers = 8 #indexed with i
num_days = 7 #indexed with j
num_shifts = 3 #indexed with k

min_workers_per_shift = 2
max_shifts_per_day = 1
min_shifts_per_worker = 5
max_night_shifts = 2

In [ ]:
# Define the model
model = ConcreteModel()

# Sets
model.workers = RangeSet(1, 8)  # Workers (i)
model.days = RangeSet(1, 7)     # Days (j)
model.shifts = RangeSet(1, 3)   # Shifts (k)

# Parameters
min_workers_per_shift = 2
max_shifts_per_day = 1
min_shifts_per_worker = 5
max_night_shifts = 2  # Assume night shift is shift 3

# Decision Variables
model.assign = Var(model.workers, model.days, model.shifts, domain=Binary)  # Binary: assigned (1) or not (0)

# Objective: Minimize the total number of shifts assigned
model.obj = Objective(expr=sum(model.assign[i, j, k] for i in model.workers for j in model.days for k in model.shifts), sense=minimize)

# Constraints
# 1. Each shift on each day must have at least `min_workers_per_shift` workers
def min_workers_per_shift_rule(model, j, k):
    return sum(model.assign[i, j, k] for i in model.workers) >= min_workers_per_shift
model.min_workers_per_shift = Constraint(model.days, model.shifts, rule=min_workers_per_shift_rule)

# 2. Each worker can work at most `max_shifts_per_day` shifts per day
def max_shifts_per_day_rule(model, i, j):
    return sum(model.assign[i, j, k] for k in model.shifts) <= max_shifts_per_day
model.max_shifts_per_day = Constraint(model.workers, model.days, rule=max_shifts_per_day_rule)

# 3. Each worker must work at least `min_shifts_per_worker` shifts in total
def min_shifts_per_worker_rule(model, i):
    return sum(model.assign[i, j, k] for j in model.days for k in model.shifts) >= min_shifts_per_worker
model.min_shifts_per_worker = Constraint(model.workers, rule=min_shifts_per_worker_rule)

# 4. Each worker can work at most `max_night_shifts` (shift 3)
def max_night_shifts_rule(model, i):
    return sum(model.assign[i, j, 3] for j in model.days) <= max_night_shifts
model.max_night_shifts = Constraint(model.workers, rule=max_night_shifts_rule)

# Solve the model
solver = SolverFactory('cbc')  # You can use 'glpk' or any other available solver
solver.solve(model)

#printing out solution (with pandas)
print("total number of shifts assigned:", model.obj())

# Display the results
print("Optimal Shift Assignment:")
for i in model.workers:
    for j in model.days:
        for k in model.shifts:
            if model.assign[i, j, k]() == 1:
                print(f"Worker {i} is assigned to day {j}, shift {k}")


total number of shifts assigned: 42.0
Optimal Shift Assignment:
Worker 1 is assigned to day 1, shift 2
Worker 1 is assigned to day 2, shift 1
Worker 1 is assigned to day 3, shift 1
Worker 1 is assigned to day 5, shift 3
Worker 1 is assigned to day 6, shift 2
Worker 1 is assigned to day 7, shift 3
Worker 2 is assigned to day 1, shift 3
Worker 2 is assigned to day 4, shift 1
Worker 2 is assigned to day 5, shift 2
Worker 2 is assigned to day 6, shift 3
Worker 2 is assigned to day 7, shift 1
Worker 3 is assigned to day 1, shift 1
Worker 3 is assigned to day 2, shift 2
Worker 3 is assigned to day 3, shift 3
Worker 3 is assigned to day 4, shift 1
Worker 3 is assigned to day 5, shift 1
Worker 4 is assigned to day 1, shift 3
Worker 4 is assigned to day 3, shift 2
Worker 4 is assigned to day 5, shift 1
Worker 4 is assigned to day 6, shift 3
Worker 4 is assigned to day 7, shift 1
Worker 5 is assigned to day 1, shift 1
Worker 5 is assigned to day 2, shift 1
Worker 5 is assigned to day 3, shift 3
